### Attempting to remove stopwords before splitting the line lyrics by Token

In [1]:
# Import libraries and upload OHCO (by line) #
import pandas as pd
import numpy as np
import os
os.chdir('/Users/nickbruno/Documents/spring_2019/DS5559/project/code')

df = pd.read_csv('artist_song_verse_line_OHCO_df.csv') # includes corpus with no duplicate song names

In [2]:
df.head()

,artist_id,song_id,verse_num,line_num,Line
0,0,0,0,0,"Look at her face, it's a wonderful face"
1,0,0,0,1,And it means something special to me
2,0,0,0,2,Look at the way that she smiles when she sees me
3,0,0,0,3,How lucky can one fellow be?
4,0,0,1,0,"She's just my kind of girl, she makes me feel ..."


In [3]:
# Create a subset of 100,000 lines #
len(df) / df.artist_id.nunique() # 2000 lines per artist

2138.6666666666665

In [4]:
artist_id_list = df.artist_id.unique().tolist()
df.artist_id.nunique() # 639 artists

639

In [9]:
import random
random.seed(69)
sub_artist = random.sample(artist_id_list, k=100)
sub_artist

[38,
 98,
 172,
 68,
 622,
 354,
 337,
 564,
 426,
 477,
 440,
 582,
 450,
 533,
 330,
 575,
 563,
 452,
 147,
 424,
 403,
 65,
 634,
 295,
 152,
 202,
 623,
 319,
 218,
 517,
 214,
 105,
 101,
 398,
 66,
 270,
 383,
 41,
 64,
 96,
 225,
 31,
 150,
 257,
 252,
 282,
 156,
 492,
 82,
 338,
 271,
 549,
 532,
 428,
 34,
 263,
 87,
 127,
 316,
 407,
 280,
 159,
 502,
 483,
 198,
 286,
 547,
 192,
 413,
 223,
 281,
 425,
 498,
 433,
 266,
 607,
 574,
 363,
 521,
 391,
 57,
 81,
 454,
 447,
 130,
 126,
 374,
 507,
 215,
 27,
 402,
 561,
 253,
 566,
 457,
 157,
 60,
 293,
 117,
 516]

In [10]:
# Subset to create smaller corpus #
subset_df = df[df.artist_id.isin(sub_artist)]

In [11]:
len(subset_df)

222192

In [12]:
subset_df = subset_df.set_index(['artist_id','song_id','verse_num','line_num'])
subset_df.head()

Line
artist_id song_id verse_num line_num                                                   
27        1826    0         0                           All I want for Christmas is you
                            1         Your the gift that's made my dreams all come true
                            2                          All I need for Christmas is here
                            3         Finding every sweet surprise wrapped up in you...
                            4                  Waiting there for me underneath the tree

In [13]:
# Create a bag of tokens #
TOKEN_PAT = r'(\W+)'
token_subset_df = subset_df.Line.str.split(TOKEN_PAT, expand=True)\
    .stack()\
    .to_frame()\
    .rename(columns={0:'token_str'})
token_subset_df.index.names = ['artist_id','song_id','verse_num','line_num','token_num']

In [14]:
token_subset_df.head()

token_str
artist_id song_id verse_num line_num token_num          
27        1826    0         0        0               All
                                     1                  
                                     2                 I
                                     3                  
                                     4              want

In [15]:
# Remove numbers and punctuation #
token_subset_df['punc'] = token_subset_df.token_str.str.match(r'^[\W_]*$').astype('int')
token_subset_df['num'] = token_subset_df.token_str.str.match(r'\d').astype('int')

In [16]:
# Create subsets where punc and num == 0
no_punc = token_subset_df[token_subset_df.punc == 0]
no_num = no_punc[no_punc.num == 0]

In [17]:
# change each word to lowercase #
no_num.token_str = no_num.token_str.str.lower()

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
# Remove stopwords #
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [19]:
final_token_df = no_num[-no_num.token_str.isin(stop)]

In [20]:
final_token_df.head(10)

token_str  punc  num
artist_id song_id verse_num line_num token_num                      
27        1826    0         0        4               want     0    0
                                     8          christmas     0    0
                            1        4               gift     0    0
                                     10              made     0    0
                                     14            dreams     0    0
                                     18              come     0    0
                                     20              true     0    0
                            2        4               need     0    0
                                     8          christmas     0    0
                            3        0            finding     0    0

In [21]:
final_token_df = final_token_df.drop(columns=['punc','num'])

In [22]:
len(final_token_df)

674695

In [23]:
# Write out to csv
final_token_df.to_csv('token_corpus_one_hundred_artists.csv')

In [24]:
df = final_token_df

### Now that we have our token table, we can use TFIDF for analysis

In [25]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/nickbruno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nickbruno/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nickbruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/nickbruno/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nickbruno/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
# Stem each word #
stemmer = nltk.stem.porter.PorterStemmer()
df['port_stem'] = df.token_str.apply(lambda x: stemmer.stem(x))

In [27]:
# Find teh word frequency
token_counts = df.port_stem.str.lower().value_counts()

In [28]:
token_counts # love, like, time

love            11581
know             8677
oh               7305
like             7125
get              5802
got              5784
go               5659
want             5468
time             5428
come             5427
babi             5059
see              4974
one              4806
make             4418
let              4345
yeah             4276
say              4266
never            4247
feel             4140
take             3908
way              3718
caus             3438
gonna            3365
back             3305
look             3285
day              3249
girl             3221
heart            3180
need             3140
night            2878
                ...  
buttonhol           1
brigitt             1
scoff               1
facism              1
owa                 1
gadget              1
menm                1
thorazin            1
snakeskin           1
michelangelo        1
fresco              1
nappi               1
necromast           1
impostor            1
chrsitma  

In [29]:
# Create a vocab table #
vocab = df.port_stem.value_counts().to_frame()\
    .reset_index()\
    .rename(columns={'index':'port_stem', 'port_stem':'n'})
vocab = vocab.sort_values('port_stem').reset_index()
vocab.index.name = 'term_id'
vocab = vocab.drop('index', 1)

In [30]:
vocab.sort_values('n', ascending=False).head(25)

,port_stem,n
term_id,,
9751,love,11581
9063,know,8677
11625,oh,7305
9521,like,7125
6648,get,5802
6881,got,5784
6802,go,5659
18319,want,5468
16984,time,5428


In [31]:
K = df
V = vocab

In [32]:
K['term_id'] = K['port_stem'].map(V.reset_index().set_index('port_stem').term_id).fillna(-1).astype('int')

In [33]:
K.head()

token_str port_stem  term_id
artist_id song_id verse_num line_num token_num                              
27        1826    0         0        4               want      want    18319
                                     8          christmas  christma     2938
                            1        4               gift      gift     6683
                                     10              made      made     9900
                                     14            dreams     dream     4828

In [34]:
OHCO = ['artist_id', 'song_id', 'verse_num', 'line_num','token_num']
SONG=OHCO[1:2]

In [35]:
# Create a bag of words per song #
BOW = K.groupby(SONG+['term_id'])['term_id'].count()
BOW.head()

song_id  term_id
1826     668        2
         1399       2
         2938       6
         3264       1
         4034       1
Name: term_id, dtype: int64

In [36]:
# Document term matrix #
DTM = BOW.unstack().fillna(0)
DTM

term_id,0,1,2,3,4,5,6,7,8,9,...,19142,19143,19144,19145,19146,19147,19148,19149,19150,19151
song_id,,,,,,,,,,,,,,,,,,,,,
1826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Now creating the TFIDF table #
alpha = .000001 # We introduce an arbitrary smoothing value
alpha_sum = alpha * V.shape[0]
TF = DTM.apply(lambda x: (x + alpha) / (x.sum() + alpha_sum), axis=1)

In [38]:
TF.head()

term_id,0,1,2,3,4,5,6,7,8,9,...,19142,19143,19144,19145,19146,19147,19148,19149,19150,19151
song_id,,,,,,,,,,,,,,,,,,,,,
1826,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,...,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08,1.922369e-08
1827,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,...,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08,2.082502e-08
1828,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,...,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09,6.801835e-09
1829,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,...,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08,1.408071e-08
1830,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,...,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09,7.193253e-09


In [39]:
N_docs = DTM.shape[0]
V['df'] = DTM[DTM > 0].count()
TFIDF = TF * np.log2(N_docs / V['df'])

In [71]:
TFIDF.head()

term_id,0,1,2,3,4,5,6,7,8,9,...,19142,19143,19144,19145,19146,19147,19148,19149,19150,19151
song_id,,,,,,,,,,,,,,,,,,,,,
1826,2.417483e-07,2.417483e-07,2.112794e-07,2.225246e-07,2.417483e-07,2.417483e-07,2.417483e-07,2.417483e-07,2.417483e-07,2.417483e-07,...,2.417483e-07,2.417483e-07,2.417483e-07,2.225246e-07,2.417483e-07,2.417483e-07,2.417483e-07,2.417483e-07,2.417483e-07,2.417483e-07
1827,2.618859e-07,2.618859e-07,2.288790e-07,2.410609e-07,2.618859e-07,2.618859e-07,2.618859e-07,2.618859e-07,2.618859e-07,2.618859e-07,...,2.618859e-07,2.618859e-07,2.618859e-07,2.410609e-07,2.618859e-07,2.618859e-07,2.618859e-07,2.618859e-07,2.618859e-07,2.618859e-07
1828,8.553674e-08,8.553674e-08,7.475609e-08,7.873491e-08,8.553674e-08,8.553674e-08,8.553674e-08,8.553674e-08,8.553674e-08,8.553674e-08,...,8.553674e-08,8.553674e-08,8.553674e-08,7.873491e-08,8.553674e-08,8.553674e-08,8.553674e-08,8.553674e-08,8.553674e-08,8.553674e-08
1829,1.770725e-07,1.770725e-07,1.547551e-07,1.629918e-07,1.770725e-07,1.770725e-07,1.770725e-07,1.770725e-07,1.770725e-07,1.770725e-07,...,1.770725e-07,1.770725e-07,1.770725e-07,1.629918e-07,1.770725e-07,1.770725e-07,1.770725e-07,1.770725e-07,1.770725e-07,1.770725e-07
1830,9.045904e-08,9.045904e-08,7.905800e-08,8.326579e-08,9.045904e-08,9.045904e-08,9.045904e-08,9.045904e-08,9.045904e-08,9.045904e-08,...,9.045904e-08,9.045904e-08,9.045904e-08,8.326579e-08,9.045904e-08,9.045904e-08,9.045904e-08,9.045904e-08,9.045904e-08,9.045904e-08


In [72]:
# Write tfidf results to a .csv #
TFIDF.to_csv('TFIDF.csv')

In [41]:
THM = -(TF * np.log2(TF))

In [43]:
# TFTH = TF.apply(lambda x: x * THM.sum(), 1)

In [45]:
V['tf_sum'] = TF.sum()
V['tf_mean'] = TF.mean()

In [46]:
V['tf_max'] = TF.max()
V['tfidf_sum'] = TFIDF.sum()

In [47]:
V['tfidf_mean'] = TFIDF.mean()
V['tfidf_max'] = TFIDF.max()

In [ ]:
# V['tfth_sum'] = TFTH.sum()
# V['tfth_mean'] = TFTH.mean()
# V['tfth_max'] = TFTH.max()

In [49]:
V['th_sum'] = THM.sum()
V['th_mean'] = THM.mean()
V['th_max'] = THM.max()

In [55]:
V.sort_values(by='n', ascending=False)

,port_stem,n,df,tf_sum,tf_mean,tf_max,tfidf_sum,tfidf_mean,tfidf_max,th_sum,th_mean,th_max
term_id,,,,,,,,,,,,
9751,love,11581,2806,112.096330,1.836441e-02,0.398312,125.686884,0.020591,0.446603,452.362715,0.074109,0.528970
9063,know,8677,3102,81.721311,1.338816e-02,0.239632,79.805459,0.013074,0.234014,391.215941,0.064092,0.493907
11625,oh,7305,1895,60.113574,9.848226e-03,0.545999,101.445089,0.016619,0.921404,259.272873,0.042476,0.530613
9521,like,7125,2619,60.099532,9.845926e-03,0.310979,73.365836,0.012019,0.379624,298.921936,0.048971,0.524034
6648,get,5802,2153,48.141725,7.886914e-03,0.366908,72.376589,0.011857,0.551612,237.874823,0.038970,0.530736
6881,got,5784,1984,47.606438,7.799220e-03,0.303394,77.186361,0.012645,0.491906,230.348738,0.037737,0.522061
6802,go,5659,2277,50.718723,8.309096e-03,0.393098,72.153504,0.011821,0.559229,251.491214,0.041201,0.529518
18319,want,5468,1884,50.987813,8.353180e-03,0.369411,86.473087,0.014167,0.626506,232.992324,0.038170,0.530733
16984,time,5428,2367,53.377285,8.744640e-03,0.242389,72.950482,0.011951,0.331272,265.424554,0.043484,0.495590


In [56]:
# Write results to .csv #
V.to_csv('V.csv')

In [58]:
# Find significant words #
TOPS = pd.DataFrame(index=range(10))
for m in ['tf','tfidf','th']:
    for n in ['mean','max']:
        key = '{}_{}'.format(m,n)
        TOPS[key] = V.sort_values(key, ascending=False).port_stem.head(10).tolist()

In [59]:
TOPS

,tf_mean,tf_max,tfidf_mean,tfidf_max,th_mean,th_max
0,love,la,love,floy,love,purpos
1,know,oh,oh,diddley,know,get
2,oh,pop,babi,allelujah,like,want
3,like,spin,want,carnag,time,ooh
4,time,head,know,phresh,oh,allelujah
5,want,woo,yeah,alberta,go,la
6,go,fa,got,fa,see,tri
7,come,bye,come,bop,come,simmer
8,see,give,like,conni,get,sky
9,get,one,time,simmer,want,see


In [60]:
def get_songs(tokens, div_names, doc_str = 'term_id', sep='', flatten=False, 
             index_only=False):
    
    if not index_only:
        docs = tokens.groupby(div_names)[doc_str]\
          .apply(lambda x: x.str.cat(sep=sep))
        docs.columns = ['doc_content']
    else:
        docs = tokens.groupby(div_names)[doc_str].apply(lambda x: x.tolist())

    if flatten:
        docs = docs.reset_index().drop(div_names, 1)
    
    return docs

In [61]:
D = get_songs(K, SONG, 'port_stem', sep=" ")\
    .to_frame().rename(columns={'port_stem':'song_str'})

In [62]:
D

,song_str
song_id,
1826,want christma gift made dream come true need c...
1827,absent mind absent mind lose key miss glove li...
1828,guy doll averag unmarri femal basic lli insecu...
1829,love bath glow rememb happen moment ago like r...
1830,found perfect love show everyth know love time...
1831,beauti spaciou sky amber wave grain purpl moun...
1832,choru hey come joe run street hey joe must new...
1833,someon hold close someon hurt deep someon sit ...
1834,say child night noth black noth white say alri...


In [63]:
D['term_count'] = DTM.sum(1).astype('int').to_frame().rename(columns={0:'term_count'})
D['tf'] = D.term_count / D.term_count.sum()

In [69]:
D.to_csv('D.csv')

### Get all Doc-Pairs

In [64]:
song_ids = D.index.tolist()
pairs = [(i,j) for i in song_ids for j in song_ids if j > i]

In [65]:
P = pd.DataFrame(pairs).reset_index(drop=True).set_index([0,1])
P.index.names = ['doc_x','doc_y']

In [67]:
def euclidean(row):
    D1 = TFIDF.loc[row.name[0]]
    D2 = TFIDF.loc[row.name[1]]
    x = (D1 - D2)**2
    y = x.sum() 
    z = np.sqrt(y)
    return z

In [68]:
P['euclidean'] = 0
P['euclidean'] = P.apply(euclidean, 1)

KeyboardInterrupt: 